# Neighborhood in the city of Canada, Toronto Neighborhoods

import libraries

In [5]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

ImportError: No module named 'folium'

using the data from 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Printing the header and the status code

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Server': 'mw1272.eqiad.wmnet', 'Accept-Ranges': 'bytes', 'Backend-Timing': 'D=118022 t=1560404737024273', 'X-Content-Type-Options': 'nosniff', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate', 'Date': 'Sun, 16 Jun 2019 06:58:59 GMT', 'X-Cache-Status': 'hit-front', 'Age': '177437', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Vary': 'Accept-Encoding,Cookie,Authorization,X-Seven', 'X-Client-IP': '169.62.147.30', 'Via': '1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1)', 'Last-Modified': 'Thu, 06 Jun 2019 10:22:24 GMT', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Length': '14903', 'Content-language': 'en', 'Set-Cookie': 'WMF-Last-Access=16-Jun-2019;Path=/;HttpOnly;secure;Expires=Thu, 18 Jul 2019 00:00:00 GMT, WMF-

Cleaning up the data

In [7]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

DATA into a dataframe

In [8]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)

(211, 3)


- custom groupby / agg to merge Neighborhoods
    - groupby PostalCode, keep the first Borough and join() Neighborhoods

In [10]:
df = df.groupby('PostalCode').agg({
        'Borough':'first', 
        'Neighborhood': ', '.join,}).reset_index()

Checking if M5A example is correct

In [6]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


- will check the df.shape

In [7]:
df.shape

(103, 3)

Reading the geo data

In [8]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

We merge two dataframes based on a PostalCode column

In [9]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [10]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [11]:
df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


Toronto map

In [12]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Analyzing Four Square API

In [13]:
CLIENT_ID = '9IU6829RKWPHYKZLBFMACHLTIIWFEFBYIUEYRVA5UM62TY9NB'
CLIENT_SECRET = '5TY80921IOC1RSX28HFEGSEWX0OFBVE5XAZLQP07464GRM3'
VERSION = '20180605' 

In [14]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [15]:
LIMIT = 100 
radius = 500 
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=3L5CRERKWPHYKZLBFMACHLTIIWFEFBYIUEYRVA5UMV3LA2MH&client_secret=3BVQTTMWIOC1RSX4TBFEGSEWX0OFBVE5XAZLQP02O0GZC2J4&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [16]:
results = requests.get(url).json()

In [17]:
# extracts venue category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']

In [12]:
nearby_venues = json_normalize(venues) 

NameError: name 'json_normalize' is not defined

In [20]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sheriff's No Frills,Grocery Store,43.741968,-79.586639
1,Subway,Sandwich Place,43.742421,-79.589471
2,Shoppers Drug Mart,Pharmacy,43.740832,-79.583347
3,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
4,The Beer Store,Beer Store,43.741694,-79.584373


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

11 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [24]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [25]:
print(venues.shape)
venues.head()

(2256, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [26]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,111,111,111,111,111,111
Downtown Toronto,1286,1286,1286,1286,1286,1286
East Toronto,121,121,121,121,121,121
East York,77,77,77,77,77,77
Etobicoke,72,72,72,72,72,72
Mississauga,10,10,10,10,10,10
North York,252,252,252,252,252,252
Queen's Park,39,39,39,39,39,39
Scarborough,89,89,89,89,89,89


In [27]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 280 unique categories.


In [28]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
onehot.shape

(2256, 280)

In [30]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.009009,0.000000,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.009009
1,Downtown Toronto,0.000000,0.000778,0.000778,0.000778,0.000778,0.000778,0.001555,0.001555,0.001555,...,0.002333,0.013219,0.002333,0.000778,0.003888,0.000000,0.006221,0.000778,0.000778,0.002333
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024793
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.003968,0.000000,0.000000,0.003968,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.003968,0.003968,0.007937,0.000000,0.000000,0.003968,0.011905,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.025641
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011173,0.000000,0.000000,0.011173,0.000000,0.005587,0.000000,0.000000,0.011173


In [31]:
grouped.shape

(11, 280)

In [32]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.06
2     Pizza Place  0.06
3            Café  0.05
4            Park  0.05


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.03
3       Bakery  0.03
4        Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4                Café  0.04


----East York----
                 venue  freq
0          Coffee Shop  0.06
1  Sporting Goods Shop  0.04
2         Burger Joint  0.04
3        Grocery Store  0.04
4                 Bank  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1  Sandwich Place  0.07
2     Coffee Shop  0.06
3        Pharmacy  0.06
4    Liquor Store  0.04


----Mississauga----
                      venue  freq
0                     Hotel   0.2
1               Coffee Shop   0.2
2  Med

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Dessert Shop,Pub,American Restaurant,Thai Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Gastropub,Pizza Place
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Park,Brewery,Yoga Studio,Pub,Pizza Place
3,East York,Coffee Shop,Park,Sporting Goods Shop,Burger Joint,Pizza Place,Grocery Store,Pharmacy,Bank,Athletics & Sports,Sushi Restaurant
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Grocery Store,Gym,Liquor Store,Fast Food Restaurant,Bakery,Convenience Store
5,Mississauga,Coffee Shop,Hotel,American Restaurant,Gym / Fitness Center,Fried Chicken Joint,Mediterranean Restaurant,Sandwich Place,Burrito Place,Dumpling Restaurant,Dog Run
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Grocery Store,Pharmacy,Park,Pizza Place,Sandwich Place
7,Queen's Park,Coffee Shop,Sushi Restaurant,Gym,Japanese Restaurant,Park,Portuguese Restaurant,Bar,Café,Italian Restaurant,Yoga Studio
8,Scarborough,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Bakery,Coffee Shop,Breakfast Spot,Playground,Sandwich Place,Bus Line,Pharmacy
9,West Toronto,Bar,Coffee Shop,Café,Bakery,Italian Restaurant,Pizza Place,Restaurant,Breakfast Spot,Diner,Park


In [35]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 1, 1, 3, 1, 4, 1, 0])

In [36]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Dessert Shop,Pub,American Restaurant,Thai Restaurant
1,Downtown Toronto,0.0,0.000778,0.000778,0.000778,0.000778,0.000778,0.001555,0.001555,0.001555,...,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Gastropub,Pizza Place
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Park,Brewery,Yoga Studio,Pub,Pizza Place
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Park,Sporting Goods Shop,Burger Joint,Pizza Place,Grocery Store,Pharmacy,Bank,Athletics & Sports,Sushi Restaurant
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Grocery Store,Gym,Liquor Store,Fast Food Restaurant,Bakery,Convenience Store
